# Problem Statement

## Q1. Linear Programming

Your start-up will face the cash requirements shown in Table 1 in the next eight quarters (positive entries represent cash needs while negative entries represent cash surpluses). The company has three borrowing possibilities.

**Table 1: Cash Flow (in Crores of INR)**

| Q1  | Q2  | Q3  | Q4   | Q5   | Q6  | Q7  | Q8   |
| --- | --- | --- | ---- | ---- | --- | --- | ---- |
| 100 | 500 | 100 | -600 | -500 | 200 | 600 | -900 |

**Borrowing Possibilities:**

*   A 2-year loan available at the beginning of Q1, with a 1% interest per quarter.
*   The other two borrowing opportunities are available at the beginning of every quarter:
    *   A 6-month loan with a 1.8% interest per quarter.
    *   A quarterly loan with a 2.5% interest for the quarter.
*   Any surplus can be invested at a 0.5% interest per quarter.

**Questions:**

a) Formulate a LP that maximizes the wealth of the company at the beginning of Q9.

b) Use built-in LP solvers to get a solution.

c) What is a good initial basic feasible point?

d) Use the revised simplex method and see if your solution matches the one from built-in solver. Explain.